In [ ]:
import numpy as np
import plotly.graph_objects as go

In [ ]:
def kalman_schlegel(duration):
    """
    Annotated version of Professor Dr. Schlegel's implementation of the kalman filter translated to python.
    """
    # Kalman filter parameters
    # Time between updates
    dt = 1
    # Standard deviation: sigma_x (sensing: measuring noise x-coordinate)
    sx = 0.001
    # Standard deviation: sigma_y (sensing: measuring noise y-coordinate)
    sy = 0.2
    # Standard deviation: sigma_v (prediction: process noise velocity)
    sv = 0.0001

    # Transition matrix (action)
    A = np.array(
        [
            [1, 0, dt, 0],
            [0, 1, 0, dt],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ]
    )
    # Noise prediction step. Describes the uncertainty in the system dynamics itself (inaccuries in vehicles motion model, random disturbances,..)
    Q = np.array(
        [
            [0, 0, 0, 0],
            [0, 0, 0, 0],
            [0, 0, sv ** 2, 0],
            [0, 0, 0, sv ** 2]
        ]
    )
    # Noise correction step
    R = np.array(
        [
            [sx ** 2, 0],
            [0, sy ** 2]
        ]
    )
    # Transition matrix (sensing). (Linear or (non-linear -> jacobian) transformation)
    C = np.array(
        [
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ]
    )
    # Initial state [px, py, vx, vy]
    x = np.array([0, 5, 1, 0])[:, np.newaxis]
    # Initial covariance matrix (produces "Einheitsmatrix" with 0.01 on diagonal)
    Sx = np.diag([0.01, 0.01, 0.01, 0.01])

    # Simulation of vehicle
    sim_sx = 0.001  # Simulated x state
    sim_sy = 0.1    # Simulated y state
    sim_sv = 0.0001 # Simulated velocity state

    # Initialize arrays for later plotting
    # Position array
    plot_pos = []
    # Measurement array
    plot_posmeas = []
    # True position of the vehicle
    plot_postrue = []

    # True position of the vehicle
    xtrue = x

    # Execute kalman filter
    for t in np.arange(0, duration + dt, dt):
        # Add noise to simulated x, y and velocity values.
        # --> Those are the "real" positions of the vehicle that are now perturbed by noise.
        processnoise = np.array(
            [
                sim_sx ** 2 * np.random.randn(),
                sim_sy ** 2 * np.random.randn(),
                sim_sv ** 2 * np.random.randn(),
                sim_sv ** 2 * np.random.randn()
            ]
        )[:, np.newaxis]
        xtrue = A @ xtrue + processnoise    # Actual position = Action model @ actual state + noise

        # Predict step
        x = A @ x                 # New actual state based on past state & Motion model
        Sx = A @ Sx @ A.T + Q     # New x_k = F * x_k-1 + Q
        # A @ Sx:                   Propagation of the current error covariance (Sx) through the system dynamics (A).
        #                           "How does the uncertainty in the current state affect the uncertainty in the next state?"
        # A @ Sx @ A.T:             Complete transformation of the error covariance through the system dynamics (A).
        #                           A is used because the cov matrix must remain symmetric to be valid.
        # A @ Sx @ A.T + Q:         Add process noise cov matrix Q to propagated uncertainty.
        #                           Accounts for uncertainty introduced by the system dynamics during the current time step.

        # Sensing activity
        measurementnoise = np.array(
            [
                sx ** 2 * np.random.randn(),
                sy ** 2 * np.random.randn()
            ]
        )[:, np.newaxis]

        y = C @ xtrue + measurementnoise # Transformation @ Actual position of the simulation + noise to perturb

        # Correct step
        H = Sx @ C.T @ np.linalg.inv(C @ Sx @ C.T + R) #    H (Kalman gain), Sx = Previous, R = Noise
        # Sx @ C.T                                          x * transposed transition matrix (measurement model)
        # C @ Sx @ C.T + R                                  transition matrix @ state matrix @ transposed state matrix + cov matrix of observational uncertainty
        # Sx @ C.T @ (C @ Sx @ C.T + R)^-1
        x = x + H @ (y - C @ x)     # Calculate error, adjust state
        Sx = Sx - H @ C @ Sx        #

        # Save some parameters for plotting later
        plot_postrue.append(xtrue[:2].flatten())
        plot_pos.append(x[:2].flatten())
        plot_posmeas.append(y.flatten())

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=np.array(plot_pos)[:, 0],
            y=np.array(plot_pos)[:, 1],
            mode='lines',
            name='Estimated vehicle position',
            line=dict(color='red')
        )
    )

    fig.add_trace(
        go.Scatter(
            x=np.array(plot_postrue)[:, 0],
            y=np.array(plot_postrue)[:, 1],
            mode='lines',
            name='Ground truth vehicle position',
            line=dict(color='green')
        )
    )

    fig.add_trace(
        go.Scatter(
            x=np.array(plot_posmeas)[:, 0],
            y=np.array(plot_posmeas)[:, 1],
            mode='markers',
            name='Measured vehicle position',
            marker=dict(color='blue', symbol='circle')
        )
    )

    fig.update_layout(
        title='Figure 1 - Vehicle Position (Measured, and Estimated)',
        xaxis_title='x axis (m)', yaxis_title='y axis (m)',
        legend=dict(orientation='h',
                    yanchor='bottom',
                    xanchor='center',
                    y=1.02,
                    x=0.5
                    ),
        showlegend=True
    )

    fig.show()


# Run the simulation
kalman_schlegel(20)
